In [2]:
# hide
from fastbook import *
from fastai.vision.all import *
import fastbook
fastbook.setup_book()

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)

In [4]:
Path.BASE_PATH = path

In [7]:
threes = (path / 'train' / '3').ls().sorted()
sevens = (path / 'train' / '7').ls().sorted()
threes

(#6131) [Path('train/3/10.png'),Path('train/3/10000.png'),Path('train/3/10011.png'),Path('train/3/10031.png'),Path('train/3/10034.png'),Path('train/3/10042.png'),Path('train/3/10052.png'),Path('train/3/1007.png'),Path('train/3/10074.png'),Path('train/3/10091.png')...]

In [8]:
im3_path = threes[1]
im3 = Image.open(im3_path)
im3

In [9]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
stacked_sevens = torch.stack(seven_tensors).float() / 255
stacked_threes = torch.stack(three_tensors).float() / 255
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28 * 28)

In [10]:
valid_3_tens = torch.stack([tensor(Image.open(o))
                            for o in (path / 'valid' / '3').ls()])
valid_3_tens = valid_3_tens.float() / 255
valid_7_tens = torch.stack([tensor(Image.open(o))
                            for o in (path / 'valid' / '7').ls()])
valid_7_tens = valid_7_tens.float() / 255
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28 * 28)
valid_y = tensor([1] * len(valid_3_tens) + [0] * len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x, valid_y))

In [11]:
train_y = tensor([1] * len(threes) + [0] * len(sevens)).unsqueeze(1)
dset = list(zip(train_x, train_y))

In [12]:
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256)
dls = DataLoaders(dl, valid_dl)

In [13]:
lr = 1.

In [14]:
def mnist_loss(predictions, targets):
    return torch.where(targets == 1, 1 - predictions, predictions).mean()

In [15]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds > 0.5) == yb
    return correct.float().mean()

In [16]:
learn = Learner(dls, nn.Linear(28 * 28, 1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [17]:
learn.fit(10, lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,-82.954834,-335.733093,0.953386,00:00
1,-309.327972,-672.049927,0.953386,00:00
2,-570.715698,-1008.366699,0.953386,00:00
3,-854.969238,-1344.683594,0.953386,00:00
4,-1152.399658,-1681.000488,0.953386,00:00
5,-1456.732910,-2017.317261,0.953386,00:00
6,-1764.437500,-2353.634033,0.953386,00:00
7,-2073.706299,-2689.950928,0.953386,00:00
8,-2383.674561,-3026.267822,0.953386,00:00
9,-2693.946777,-3362.584473,0.953386,00:00
